In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
from torchvision import datasets, transforms, models
from torch.utils.data import DataLoader
from torchvision import models
# from torchsummary import summary
from torchinfo import summary
import wandb
import os

In [2]:
# =======================
# STEP 0: Initialize wandb
# =======================
wandb.init(project="MobileNet-flowers", config={
    "epochs": 50,
    "batch_size": 16,
    "learning_rate": 0.001,
    "architecture": "MobileNet",
    "pretrained": True,
    "input_size": 224
})

# Shortcut to config values
config = wandb.config

wandb: [wandb.login()] Loaded credentials for https://api.wandb.ai from C:\Users\Lenovo\_netrc.
wandb: Currently logged in as: yash-ingle002 (yash-ingle002-sardar-vallabhbhai-national-institute-of-t) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


In [3]:
from torchvision import datasets, transforms
from torch.utils.data import DataLoader, random_split


# Transforms

train_transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406],
                         [0.229, 0.224, 0.225])
])

val_transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406],
                         [0.229, 0.224, 0.225])
])


# Dataset path (YOUR CURRENT LOCATION)

data_dir = r"C:\Users\Lenovo\OneDrive\Desktop\deep Learning\Dataset\flowers"

# Load full dataset
full_dataset = datasets.ImageFolder(root=data_dir)


# Train / Validation Split

train_size = int(0.8 * len(full_dataset))
val_size = len(full_dataset) - train_size

train_dataset, val_dataset = random_split(
    full_dataset, [train_size, val_size]
)

# Assign transforms AFTER split (important!)
train_dataset.dataset.transform = train_transform
val_dataset.dataset.transform = val_transform


# DataLoaders

train_loader = DataLoader(
    train_dataset,
    batch_size=config.batch_size,
    shuffle=True
)

val_loader = DataLoader(
    val_dataset,
    batch_size=config.batch_size,
    shuffle=False
)


In [4]:
# ===========================
# STEP 2: Load Pretrained MobileNetV2
# ===========================
from torchvision.models import MobileNet_V2_Weights

# Load pretrained MobileNetV2
model = models.mobilenet_v2(weights=MobileNet_V2_Weights.DEFAULT)

# Replace the final classifier layer for 5-class flower classification
model.classifier[1] = nn.Linear(model.last_channel, 5)

# Freeze all parameters except the final layer
for param in model.parameters():
    param.requires_grad = False
for param in model.classifier[1].parameters():
    param.requires_grad = True

# Move model to device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)

# Watch the model's weights and gradients with wandb
wandb.watch(model, log="all", log_freq=10)


Downloading: "https://download.pytorch.org/models/mobilenet_v2-7ebf99e0.pth" to C:\Users\Lenovo/.cache\torch\hub\checkpoints\mobilenet_v2-7ebf99e0.pth
100%|██████████| 13.6M/13.6M [00:08<00:00, 1.76MB/s]


In [5]:
# ===================
# STEP 3: Loss & Optimizer
# ===================

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=config.learning_rate)

In [6]:
def train_model(model, criterion, optimizer, train_loader, val_loader, epochs=10):
    for epoch in range(epochs):
        model.train()
        train_correct = 0
        train_total = 0
        running_loss = 0.0

        print(f"\nEpoch {epoch + 1}/{epochs}")
        print("-" * 30)

        for i, (images, labels) in enumerate(train_loader):
            images, labels = images.to(device), labels.to(device)

            optimizer.zero_grad()
            outputs = model(images)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            running_loss += loss.item()
            _, preds = torch.max(outputs, 1)
            batch_correct = (preds == labels).sum().item()
            train_correct += batch_correct
            train_total += labels.size(0)

            # Print every 10 batches
            if (i + 1) % 10 == 0:
                batch_acc = batch_correct / labels.size(0)
                print(f"[Batch {i+1}/{len(train_loader)}] Loss: {loss.item():.4f}, Batch Acc: {batch_acc:.4f}")

        train_acc = train_correct / train_total
        wandb.log({"epoch": epoch + 1, "train_loss": running_loss, "train_accuracy": train_acc})
        print(f"Epoch {epoch+1} Summary - Loss: {running_loss:.4f}, Train Accuracy: {train_acc:.4f}")

        # Validation
        model.eval()
        val_correct = 0
        val_total = 0
        with torch.no_grad():
            for images, labels in val_loader:
                images, labels = images.to(device), labels.to(device)
                outputs = model(images)
                _, preds = torch.max(outputs, 1)
                val_correct += (preds == labels).sum().item()
                val_total += labels.size(0)

        val_acc = val_correct / val_total
        wandb.log({"epoch": epoch + 1, "val_accuracy": val_acc})
        print(f"Validation Accuracy: {val_acc:.4f}")


In [7]:
# ===================
# Train the model
# ===================
train_model(model, criterion, optimizer, train_loader, val_loader, epochs=config.epochs)


Epoch 1/50
------------------------------
[Batch 10/216] Loss: 1.4825, Batch Acc: 0.5000
[Batch 20/216] Loss: 1.1655, Batch Acc: 0.7500
[Batch 30/216] Loss: 1.0663, Batch Acc: 0.6875
[Batch 40/216] Loss: 0.9609, Batch Acc: 0.7500
[Batch 50/216] Loss: 1.0372, Batch Acc: 0.6250
[Batch 60/216] Loss: 0.7436, Batch Acc: 0.9375
[Batch 70/216] Loss: 1.1882, Batch Acc: 0.4375
[Batch 80/216] Loss: 0.7585, Batch Acc: 0.8125
[Batch 90/216] Loss: 0.9848, Batch Acc: 0.6250
[Batch 100/216] Loss: 0.4569, Batch Acc: 0.9375
[Batch 110/216] Loss: 0.9829, Batch Acc: 0.6250
[Batch 120/216] Loss: 0.6307, Batch Acc: 0.8125
[Batch 130/216] Loss: 0.7744, Batch Acc: 0.8125
[Batch 140/216] Loss: 0.3895, Batch Acc: 0.9375
[Batch 150/216] Loss: 0.4708, Batch Acc: 0.9375
[Batch 160/216] Loss: 0.4461, Batch Acc: 0.9375
[Batch 170/216] Loss: 0.5041, Batch Acc: 0.8750
[Batch 180/216] Loss: 0.4574, Batch Acc: 0.8750
[Batch 190/216] Loss: 0.4207, Batch Acc: 0.8750
[Batch 200/216] Loss: 0.8550, Batch Acc: 0.6875
[Batch